In [ ]:
#load images from folder and train a neural network to recognize faces
#uses a pre-trained model from keras

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import pickle
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
#load data from folder
DATADIR = "faces" #Replace with path to folder containing images if not in same folder as this file
CATEGORIES = ["angry", "happy", "neutral", "sad"]

#resize images to 50x50
IMG_SIZE = 50

#load images from folder

training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) #path to folder
        class_num = CATEGORIES.index(category) #assign number to each category
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE) #convert image to grayscale
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) #resize image
                training_data.append([new_array, class_num]) #add image and class number to training data
            except Exception as e:
                pass
    return training_data

In [ ]:
from sklearn.model_selection import train_test_split
training_data = create_training_data()

#shuffle data
random.shuffle(training_data)

#split data into features and labels
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

#reshape data
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

#split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#save data
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
#load data
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

#normalize data
X = X/255.0

#build model
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4))

#compile model
model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [ ]:
#train model
epochs=100
history = model.fit(
  X_train,
  y_train,
  epochs=epochs,
  validation_split=0.2
)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
check = 0
predictions = model.predict([X_test])
acc = model.evaluate(X_test, y_test)
score = tf.nn.softmax(predictions[check])

#show x_test[0] image
#plt.imshow(X_test[check], cmap=plt.cm.binary)
print("This image most likely belongs to {} with a {:.2f} percent confidence.".format(CATEGORIES[np.argmax(score)], 100 * np.max(score)))
print("This image is actualy {}.".format(CATEGORIES[y_test[check]]))